<a href="https://colab.research.google.com/github/HwayoungYoon/major-predict-test-duksung/blob/master/Model_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 환경설정 및 라이브러리 import

In [ ]:
# Google Drive와 Colab 연동
# 실행 후 나오는 URL 접속하여 authorization code 받기
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive/predict

/content/gdrive/My Drive/predict


In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier, plot_importance
import time
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# 1. 데이터 준비

In [ ]:
# data 입력 및 확인
original_df = pd.read_csv("data_merge.csv")
original_df.head()

,STUDENT_ID,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF,AR_2016,AR_2017,AR_2018,AR_2019,EVERY_TIME,MAJOR_SCORE
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1,15.35,19.85,14.4,10.5,0.111,965.750667
1,2,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1,15.35,19.85,14.4,10.5,0.111,737.820000
2,3,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1,15.35,19.85,14.4,10.5,0.111,974.641212
3,4,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1,15.35,19.85,14.4,10.5,0.111,586.063333
4,5,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1,15.35,19.85,14.4,10.5,0.111,885.163333


In [ ]:
# 필요없는 column 'STUDENT_ID', 'AR_2016', 'AR_2017', 'AR_2018', 'AR_2019', 'EVERY_TIME', 'MAJOR_SCORE' 삭제 및 확인
del original_df['STUDENT_ID']
del original_df['AR_2016']
del original_df['AR_2017']
del original_df['AR_2018']
del original_df['AR_2019']
del original_df['EVERY_TIME']
del original_df['MAJOR_SCORE']
original_df.head()

,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR,PF
0,Art&Design,동양화,18,12,4.2310,4.2098,300,1,1
1,Art&Design,동양화,18,18,4.1486,4.1206,50,1,1
2,Art&Design,동양화,18,15,4.1206,4.1066,300,1,1
3,Art&Design,동양화,18,18,4.1021,4.0617,300,0,1
4,Art&Design,동양화,18,18,4.0589,4.0509,200,1,1


In [ ]:
# column 순서 변경
original_df = original_df[['PF', 'COLLEGE', 'MAJOR_NAME', 'CREDIT_1', 'CREDIT_2', 'GPA_1', 'GPA_2',
       'EXTRA_ACTIVITY', 'COMPLETE_MAJOR']]
original_df.head()

,PF,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1
1,1,Art&Design,동양화,18,18,4.1486,4.1206,50,1
2,1,Art&Design,동양화,18,15,4.1206,4.1066,300,1
3,1,Art&Design,동양화,18,18,4.1021,4.0617,300,0
4,1,Art&Design,동양화,18,18,4.0589,4.0509,200,1


In [ ]:
# csv 파일로 저장
original_df.to_csv('dataframe_for_prediction.csv', index=False)

In [ ]:
# 저장된 파일 확인
original_df = pd.read_csv('dataframe_for_prediction.csv')
original_df

,PF,COLLEGE,MAJOR_NAME,CREDIT_1,CREDIT_2,GPA_1,GPA_2,EXTRA_ACTIVITY,COMPLETE_MAJOR
0,1,Art&Design,동양화,18,12,4.2310,4.2098,300,1
1,1,Art&Design,동양화,18,18,4.1486,4.1206,50,1
2,1,Art&Design,동양화,18,15,4.1206,4.1066,300,1
3,1,Art&Design,동양화,18,18,4.1021,4.0617,300,0
4,1,Art&Design,동양화,18,18,4.0589,4.0509,200,1
...,...,...,...,...,...,...,...,...,...
1999,0,글로벌융합,의상디자인,18,18,1.3427,1.3293,300,1
2000,0,글로벌융합,의상디자인,15,18,1.2970,1.2572,300,1
2001,0,글로벌융합,의상디자인,18,18,1.2286,1.1680,0,1
2002,0,글로벌융합,의상디자인,18,18,1.1256,1.0741,300,1


In [ ]:
# 전공명 추출
temp = original_df[['MAJOR_NAME']]
temp

,MAJOR_NAME
0,동양화
1,동양화
2,동양화
3,동양화
4,동양화
...,...
1999,의상디자인
2000,의상디자인
2001,의상디자인
2002,의상디자인


In [ ]:
# 추출된 전공명 확인
temp = temp.drop_duplicates(subset=list(temp.columns))
temp

,MAJOR_NAME
0,동양화
25,서양화
64,시각디자인
111,실내디자인
150,텍스타일디자인
188,컴퓨터공학
260,IT미디어공학
340,사이버보안
405,소프트웨어
470,바이오공학


# 2. 예측

## 1) 예측 준비

In [ ]:
# 라이브러리 import
from xgboost import XGBClassifier, plot_importance
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, f1_score, accuracy_score
from tqdm import tqdm

In [ ]:
# 클래스 생성
class CRpredict:
    def __init__(self, origin_df, MAJOR_NAME, CREDIT_1, CREDIT_2, 
                 GPA_1, GPA_2, EXTRA_ACTIVITY, COMPLETE_MAJOR):
        
        self.origin_df = origin_df
        self.MAJOR_NAME = str(MAJOR_NAME)
        self.CREDIT_1 = int(CREDIT_1)
        self.CREDIT_2 = int(CREDIT_2)
        self.GPA_1 = float(GPA_1)
        self.GPA_2 = float(GPA_2)
        self.EXTRA_ACTIVITY = int(EXTRA_ACTIVITY)
        self.COMPLETE_MAJOR = int(COMPLETE_MAJOR)

    def get_dataframe(self):
        new_df = self.origin_df[self.origin_df['MAJOR_NAME']==self.MAJOR_NAME]
        return new_df
    
    def prediction(self):
        new_df = self.origin_df[self.origin_df['MAJOR_NAME']==self.MAJOR_NAME]
        
        # INPUT 값으로 데이터프레임 생성
        data = {'CREDIT_1':[self.CREDIT_1], 'CREDIT_2':[self.CREDIT_2], 'GPA_1':[self.GPA_1], 'GPA_2':[self.GPA_2], 'EXTRA_ACTIVITY':[self.EXTRA_ACTIVITY], 'COMPLETE_MAJOR':[self.COMPLETE_MAJOR]}

        df_input = pd.DataFrame(data, columns = ['CREDIT_1', 'CREDIT_2', 'GPA_1', 'GPA_2', 'EXTRA_ACTIVITY', 'COMPLETE_MAJOR'])
    
        self.dataframe = df_input
        
        X_train = new_df[list(new_df)[3:]]
        y_train = new_df['PF']
        X_test = df_input
    
        model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)
    
        # Fitting
        start = time.time()
        model.fit(X_train, y_train)
        time_spent = time.time()-start

        # 예측
        prediction = model.predict(X_test)
        if prediction == 1:
            result = '선발 가능'
        elif prediction == 0:
            result = '선발 불가능'
            
            
        # 2학기 평점별로 전공 선발 여부 예측하기
        optimum_gpa = self.GPA_1*10000 + 100
        passornot = []
        for i in range(int(self.GPA_1*1000), 0, -100):
            optimum_gpa -= 100
            df_temp = df_input.copy()
            df_temp['GPA_2'] = df_temp['GPA_2'].replace(self.GPA_2,i/10000)

            # 예측
            prediction_2 = model.predict(df_temp)
            passornot.append([prediction_2, optimum_gpa])
            self.optimum_gpa = optimum_gpa + 1
            self.passornot = passornot
            
            if prediction_2[0] == 0:
                break
            elif prediction_2[0] == 1:
                pass
        return result

## 2) 예측 위한 데이터 입력

In [ ]:
predict_1st = CRpredict(original_df, '경영학', 18, 18, 3.56, 4.0, 200, 1)

'''
parametor 순서
self, origin_df, MAJOR_NAME, CREDIT_1, CREDIT_2, GPA_1, GPA_2, EXTRA_ACTIVITY, COMPLETE_MAJOR
'''

'\nparametor 순서\nself, origin_df, MAJOR_NAME, CREDIT_1, CREDIT_2, GPA_1, GPA_2, EXTRA_ACTIVITY, COMPLETE_MAJOR\n'

## 3) 전공 선발 여부 확인 : prediction()

In [ ]:
predict_1st.prediction()

'선발 가능'

## 4) 추천 2학기 목표 평점 : optimum_gpa

In [ ]:
predict_1st.optimum_gpa/10000

3.2101